In [ ]:
from BinaryFileUnpack import BinaryFileUnpack
import numpy as np

Initializing BinaryFileUnpack object

In [ ]:
# Replace fp with necessary path
fp = 'tests\cold-water-2in\X64cm\ColdWater_X64cm_Y58cm-20221021-19-02-48.bin'
analysis = BinaryFileUnpack(fp)
num_sens = analysis.num_sens
analysis.header_info

Forming Spectral Analysis

In [ ]:
# Pressure
Pxx = analysis.spectra(analysis.P)
# Temperature
Txx = analysis.spectra(analysis.T)

Graphing in Matplotlib (static)

In [ ]:
sta = 0; end = len(analysis.time)/analysis.fs
s = int(analysis.fs*sta); e = int(analysis.fs*end)

# Pressure and Temperature Data
analysis.plot_static(analysis.time[s:e], analysis.P[:, s:e], "time (s)", "Pressure (bar)", plots_shape=(num_sens, 1), color='b')
analysis.plot_static(analysis.time[s:e], analysis.T[:, s:e], "time (s)", r"Temperature ($^{\circ}C$)", plots_shape=(num_sens, 1), color='r')

# Spectral Analysis
analysis.plot_static(Pxx[0], Pxx[1], "Frequency (Hz)", "Power (dB)", plots_shape=(num_sens//2, 2), color='b', x_axis_type='log')
analysis.plot_static(Txx[0], Txx[1], "Frequency (Hz)", "Power (dB)", plots_shape=(num_sens//2, 2), color='r', x_axis_type='log')

Graphing in Bokeh (Interactive)

In [ ]:
# Pressure and Temperature Data
res = 1
analysis.plot_interactive(analysis.time[::res], analysis.P[:, ::res], "Time (s)", "Pressure (bar)", (num_sens, 1), color='blue')
analysis.plot_interactive(analysis.time[::res], analysis.T[:, ::res], "Time (s)", "Temperature (deg C)", (num_sens, 1), color='red')

# Spectral Analysis
analysis.plot_interactive(Pxx[0, :, ::res], Pxx[1, :, ::res], "Frequency (Hz)", "Power (dB)-Pressure", plots_shape=(num_sens//2, 2), color='blue', x_axis_type='log')
analysis.plot_interactive(Txx[0, :, ::res], Txx[1, :, ::res], "Frequency (Hz)", "Power (dB)-Temperature", plots_shape=(num_sens//2, 2), color='red', x_axis_type='log')